## The simpliest usage example of py_boost

### Installation (if needed)

**Note**: replace cupy-cuda110 with your cuda version!!!

In [1]:
# !pip install cupy-cuda110 py-boost

### Imports

In [1]:
import os
# Optional: set the device to run
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

os.makedirs('../data', exist_ok=True)

import joblib
from sklearn.datasets import make_regression

# simple case - just one class is used
from py_boost import GradientBoosting, CrossValidation

### Generation of dummy regression data

In [157]:
%%time
X, y = make_regression(150000, 100, n_targets=10, random_state=42)
X_test, y_test = X[:50000], y[:50000]
X, y = X[-50000:], y[-50000:]

CPU times: user 2.29 s, sys: 1.37 s, total: 3.66 s
Wall time: 798 ms


In [223]:
from py_boost.gpu.utils import *
from py_boost.gpu.tree import *
from py_boost.gpu.base import Ensemble

def cluster_grow_tree(tree, group, arr, grad, hess, row_indexer, col_indexer, params):
    # create gh
    n_out = grad.shape[1]
    gh = cp.concatenate((grad, hess), axis=1)
    out_indexer = cp.arange(gh.shape[1], dtype=cp.uint64)

    # init nodes with single zero node
    unique_nodes = np.zeros(1, dtype=np.int32)
    # count unique nodes in active rows
    nodes_count = cp.ones(1, dtype=cp.uint64) * row_indexer.shape[0]
    # nodes for all rows
    nodes = cp.zeros(arr.shape[0], dtype=cp.int32)
    # index of node in unique array
    node_indexes = nodes
    prev_hist, small_index, big_index = [None] * 3

    for niter in range(params['max_depth']):

        nnodes = len(unique_nodes)
        gh_hist = histogram(arr, gh, node_indexes,
                            col_indexer=col_indexer,
                            row_indexer=row_indexer,
                            out_indexer=out_indexer,
                            nnodes=nnodes,
                            max_bins=params['max_bin'],
                            prev_hist=prev_hist,
                            small_index=small_index,
                            big_index=big_index)

        # assume hess is the last output

        hist, counts = gh_hist[:-1], gh_hist[-1]
        total = hist[..., :1, -1:]
        curr = total.min(axis=0)
        gain = cp.zeros(hist.shape[1:] + (2,), dtype=cp.float32)
        
        # NAN to left
        gain[..., 0] = curr - hist.min(axis=0) - (total - hist).min(axis=0)
        gain[..., 0] *= cp.minimum(counts, counts[..., -1:] - counts) >= params['min_data_in_leaf']
        
        # NAN to right
        gain[..., 1] = curr - (hist - hist[..., :1]).min(axis=0) - (total - hist + hist[..., :1]).min(axis=0)
        gain[..., 1] *= cp.minimum(counts - counts[..., :1:], counts[..., -1:] - counts + counts[..., :1]) >= params['min_data_in_leaf']

        best_feat, best_gain, best_split, best_nan_left = get_best_split(gain, col_indexer)

        # move to CPU and apply min_gain_to_split condition
        unique_nodes, new_nodes_id, best_feat, best_gain, best_split, best_nan_left, is_valid_node = \
            get_cpu_splitters(unique_nodes, best_feat, best_gain, best_split, best_nan_left,
                              params['min_gain_to_split'])
        # if all nodes are not valid to split - exit
        if len(unique_nodes) == 0:
            break
        # write node info to the Tree
        tree.set_nodes(group, unique_nodes, new_nodes_id, best_feat, best_gain, best_split, best_nan_left)
        # get args back on gpu
        split_args, unique_nodes = get_gpu_splitters(unique_nodes, new_nodes_id,
                                                     best_feat, best_split, best_nan_left)

        # perform split for train set
        nodes, node_indexes = make_split(nodes, arr, *split_args, return_pos=True)

        # update info for the next step
        if niter < (params['max_depth'] - 1):
            # update counts
            nodes_count = cp.zeros((unique_nodes.shape[0] + 1,), dtype=np.uint64)
            nodes_count.scatter_add(node_indexes[row_indexer], 1)
            nodes_count = nodes_count[:-1]

            cpu_counts = nodes_count.get()

            # remove unused rows from indexer
            if cpu_counts.sum() < row_indexer.shape[0]:
                row_indexer = row_indexer[isin(nodes, split_args[1].ravel(), index=row_indexer)]

            # save histogram for the subs trick
            prev_hist, small_index, big_index = get_prev_hist(cpu_counts,
                                                              gh_hist, cp.asarray(is_valid_node))

    return nodes



class ClusterTreeBuilder:
    """Tree builder for early stopping clusters"""

    def __init__(self, borders,
                 **tree_params
                 ):
        """

        Args:
            borders: list of np.ndarray, actual split borders for quantized features
            **tree_params: other tree building parameters
        """
        self.borders = borders

        self.params = {**{

            'max_bin': 256,
            'max_depth': 6,
            'min_data_in_leaf': 10,
            'min_gain_to_split': 0

        }, **tree_params}


    def build_tree(self, X, y):
        """Build tree

        Args:
            X: cp.ndarray, quantized feature matrix
            y: cp.ndarray, loss path matrix


        Returns:
            tree, Tree, constructed tree
        """

        col_indexer = cp.arange(X.shape[1], dtype=cp.uint64)
        row_indexer = cp.arange(X.shape[0], dtype=cp.uint64)
        max_nodes = int((2 ** np.arange(self.params['max_depth'] + 1)).sum())
        tree = Tree(max_nodes, y.shape[1], 1)
        # grow single group of the tree and get nodes index
        cluster_grow_tree(tree, 0, X, y, cp.ones((y.shape[0], 1), dtype=cp.float32),
                          row_indexer, col_indexer, self.params)

        tree.set_borders(self.borders)
        tree.set_leaves()
        tree.set_node_values(np.zeros((max_nodes, 1) , dtype=np.float32), np.zeros((1, ) , dtype=np.uint64))

        return tree
    
class ClusterCandidates(Ensemble):
    
    def __init__(self, depth_range=list(range(1, 7)), min_data_in_leaf=100, max_bin=256, quant_sample=100000):
        
        super().__init__()
        
        self.depth_range = depth_range
        self.min_data_in_leaf = min_data_in_leaf
        self.max_clust = 2 ** max(depth_range)
        self.max_bin = max_bin
        self.quant_sample = quant_sample
        
    def fit(self, X, y):
        
        X, y, sample_weight, eval_sets = validate_input(X, y, None, [])
        mempool = cp.cuda.MemoryPool()
        with cp.cuda.using_allocator(allocator=mempool.malloc):
            X_enc, max_bin, borders, eval_enc = quantize_train_valid(X, eval_sets, self.max_bin, self.quant_sample)
            
            self.fit_quantized(X_enc, y, max_bin, borders)
        mempool.free_all_blocks()

        return self
    
    def fit_quantized(self, X_enc, y, max_bin, borders):
        
        y = cp.array(y, order='C', dtype=cp.float32)
        X_cp = pad_and_move(X_enc)
        self.models = []
        
        for d in self.depth_range:
            builder = ClusterTreeBuilder(borders, max_depth=d, min_data_in_leaf=500, max_bin=max_bin)
            self.models.append(builder.build_tree(X_cp, y))
            
        self.base_score = np.zeros((1, ), dtype=np.float32)
            
        return self
            

In [224]:
y_ = np.abs(y)

# X, y, sample_weight, eval_sets = validate_input(X, y)
# X_enc, max_bin, borders, eval_enc = quantize_train_valid(X, eval_sets, 255, 10000)
# X_cp = pad_and_move(X_enc)
# y_ = cp.array(y_, order='C', dtype=cp.float32)

In [237]:
%%time
clusters = ClusterCandidates(min_data_in_leaf=10)
clusters.fit(X, y_)

CPU times: user 1.45 s, sys: 16 ms, total: 1.47 s
Wall time: 184 ms


In [238]:
clusters.models[0].feats

array([[91, -1, -1]])

In [239]:
clusters.models[0].val_splits

array([[1.4471349, 0.       , 0.       ]], dtype=float32)

In [240]:
clusters.predict_leaves(X)[..., 0].T.max(axis=0)

array([ 1,  3,  7, 12, 14, 16], dtype=uint32)

In [241]:
leaves = tree.predict_leaf(cp.array(X, dtype=cp.float32))

In [193]:
leaves.max()

array(17, dtype=uint32)

In [194]:
cp.unique(leaves, return_counts=True)

(array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
        17], dtype=uint32),
 array([  731,   612,   815,   522,   699,   598,   912,   691,   513,
          541,   605,   528,   608,   506,   836,   742, 37867,  1674]))

### Training a GBDT model

The only argument required here is a loss function. It, together with the input target shape, determines the task type. The loss function can be passed as a Loss instance or using a string alias:

* ***'mse'*** for the regression/multitask regression
* ***'msle'*** for the regression/multitask regression
* ***'bce'*** for the binary/multilabel classification
* ***'crossentropy'*** for the multiclassification

Training is simply done by calling the .fit metod. Possible argumentsare the following:

* ***'X'*** 
* ***'y'*** 
* ***'sample_weight'*** 
* ***'eval_sets'***  
A validation set is passed as a list of dicts with possible keys ['X', 'y', 'sample_weight']. Note: if multiple valid sets are passed, the best model is selected using the last one.

#### The example below illustrates how to train a simple regression task.

In [4]:
%%time
model = GradientBoosting('mse')

model.fit(X, y[:, 0], eval_sets=[{'X': X_test, 'y': y_test[:, 0]},])

[07:46:02] Stdout logging level is INFO.
[07:46:02] GDBT train starts. Max iter 100, early stopping rounds 100
[07:46:03] Iter 0; Sample 0, rmse = 173.67502218528736; 
[07:46:03] Iter 10; Sample 0, rmse = 133.1954913545181; 
[07:46:03] Iter 20; Sample 0, rmse = 107.86651497941278; 
[07:46:03] Iter 30; Sample 0, rmse = 90.08231837066428; 
[07:46:03] Iter 40; Sample 0, rmse = 76.44512114655686; 
[07:46:03] Iter 50; Sample 0, rmse = 65.61043014181283; 
[07:46:03] Iter 60; Sample 0, rmse = 56.801627968047306; 
[07:46:03] Iter 70; Sample 0, rmse = 49.57726379316838; 
[07:46:03] Iter 80; Sample 0, rmse = 43.603736430179794; 
[07:46:03] Iter 90; Sample 0, rmse = 38.69769943824404; 
[07:46:03] Iter 99; Sample 0, rmse = 34.99192512700217; 
CPU times: user 4.28 s, sys: 740 ms, total: 5.02 s
Wall time: 3.69 s


In [5]:
%%time
model = GradientBoosting('mse')
cv = CrossValidation(model)

oof = cv.fit_predict(X, y[:, 0], stratify=False)
test_pred = cv.predict(X_test)

[07:46:04] Stdout logging level is INFO.
[07:46:04] GDBT train starts. Max iter 100, early stopping rounds 100
[07:46:04] Iter 0; Sample 0, rmse = 172.9149002108474; 
[07:46:04] Iter 10; Sample 0, rmse = 132.55210391589216; 
[07:46:04] Iter 20; Sample 0, rmse = 107.23296689918294; 
[07:46:04] Iter 30; Sample 0, rmse = 89.55113466574534; 
[07:46:04] Iter 40; Sample 0, rmse = 76.07330856675229; 
[07:46:04] Iter 50; Sample 0, rmse = 65.31944162376402; 
[07:46:04] Iter 60; Sample 0, rmse = 56.69942793984432; 
[07:46:04] Iter 70; Sample 0, rmse = 49.5628290696496; 
[07:46:04] Iter 80; Sample 0, rmse = 43.681215651498924; 
[07:46:04] Iter 90; Sample 0, rmse = 38.90670685220184; 
[07:46:04] Iter 99; Sample 0, rmse = 35.28539972019422; 
[07:46:05] Stdout logging level is INFO.
[07:46:05] GDBT train starts. Max iter 100, early stopping rounds 100
[07:46:05] Iter 0; Sample 0, rmse = 177.05438336288094; 
[07:46:05] Iter 10; Sample 0, rmse = 135.3641459583856; 
[07:46:05] Iter 20; Sample 0, rmse =

### Traininig a GBDT model in a multiregression case

Each of built-in loss functions has its own default metric, so metric definition is optional. 
If you need to specify the evaluation metric, you can pass a Metric instance or use a string alias.

#### Default metrics:

* ***'rmse'*** is the default for the ***'mse'*** loss
* ***'rmsle'*** is the default for the  ***'msle'*** loss
* ***'bce'*** is the default for the ***'bce'*** loss
* ***'crossentropy'*** is the default for the ***'crossentropy'*** loss

#### Non-default metrics:

* ***'r2'*** for the regression/multitask regression
* ***'auc'*** for the binary classification
* ***'accuracy'*** for any classification task
* ***'precision'*** for any classification task
* ***'recall'*** for any classification task
* ***'f1'*** for any classification task

It is possible to specify other common GBDT hyperparameters as shown below.

#### The following example demonstrates how to train a model for a multioutput regression task (no extra definition needed to switch the task to multioutput one, you just need to pass a multidimensional target).

In [6]:
%%time
model = GradientBoosting('mse', 'r2_score',
                         ntrees=1000, lr=.01, verbose=100, es=200, lambda_l2=1,
                         subsample=.8, colsample=.8, min_data_in_leaf=10, min_gain_to_split=0, 
                         max_bin=256, max_depth=6)

model.fit(X, y, eval_sets=[{'X': X_test, 'y': y_test},])

[07:46:36] Stdout logging level is INFO.
[07:46:36] GDBT train starts. Max iter 1000, early stopping rounds 200
[07:46:37] Iter 0; Sample 0, R2_score = 0.008384933833777941; 
[07:46:39] Iter 100; Sample 0, R2_score = 0.5168161670121248; 
[07:46:41] Iter 200; Sample 0, R2_score = 0.7243374908930145; 
[07:46:43] Iter 300; Sample 0, R2_score = 0.8327434907624806; 
[07:46:45] Iter 400; Sample 0, R2_score = 0.894976532253023; 
[07:46:47] Iter 500; Sample 0, R2_score = 0.9320361103269559; 
[07:46:49] Iter 600; Sample 0, R2_score = 0.9546979928604653; 
[07:46:51] Iter 700; Sample 0, R2_score = 0.9687510714926756; 
[07:46:54] Iter 800; Sample 0, R2_score = 0.9776249822188479; 
[07:46:56] Iter 900; Sample 0, R2_score = 0.9833214913041353; 
[07:46:58] Iter 999; Sample 0, R2_score = 0.9870160617953221; 
CPU times: user 20.4 s, sys: 2.74 s, total: 23.2 s
Wall time: 21.8 s


In [154]:
from py_boost.gpu.losses import BCELoss, BCEMetric
import cupy as cp
import numpy as np

class BCENanMetric(BCEMetric):
    
    alias = 'score'
    
    def __call__(self, y_true, y_pred, sample_weight=None):
        
        mask = ~cp.isnan(y_true)
        y_true = cp.where(mask, y_true, 0)
        err = self.error(y_true, y_pred)
        
        shape = err.shape
        assert shape[0] == y_true.shape[0], 'Error shape should match target shape at first dim'

        if len(shape) == 1:
            err = err[:, cp.newaxis]

        if sample_weight is None:
            return (err * mask).sum() / mask.sum()
        
        sw = sample_weight[mask]
        
        err = ((err * mask).sum(axis=1, keepdims=True) / mask.sum(axis=1, keepdims=True) * sw).sum() / sw.sum()
        return err
        

class BCENanLoss(BCELoss):
    
    def base_score(self, y_true):
        means = cp.clip(cp.nanmean(y_true, axis=0), self.clip_value, 1 - self.clip_value)
        return cp.log(means / (1 - means))
    
    def get_grad_hess(self, y_true, y_pred):
        
        mask = ~cp.isnan(y_true)
        grad, hess = super().get_grad_hess(cp.where(mask, y_true, 0), y_pred)
        grad = grad * mask
        hess = hess * mask
        
        return grad, hess
    

#model = GradientBoosting(BCENanLoss(), BCENanMetric()....)
    

In [155]:
sl = np.random.rand(*y.shape) > 0.5
y_nan = np.where(sl, y > y.mean(), np.nan)

sl = np.random.rand(*y_test.shape) > 0.5
y_test_nan = np.where(sl, y_test > y_test.mean(), np.nan)


In [156]:
model = GradientBoosting(BCENanLoss(), BCENanMetric(),
                         ntrees=1000, lr=.01, verbose=100, es=200, lambda_l2=1,
                         subsample=.8, colsample=.8, min_data_in_leaf=10, min_gain_to_split=0, 
                         max_bin=256, max_depth=6)

model.fit(X, y_nan, eval_sets=[{'X': X_test, 'y': y_test_nan},])

[10:22:29] Stdout logging level is INFO.
[10:22:29] GDBT train starts. Max iter 1000, early stopping rounds 200
[10:22:29] Iter 0; Sample 0, score = 0.6901909016192641; 
[10:22:31] Iter 100; Sample 0, score = 0.49825635116672295; 
[10:22:33] Iter 200; Sample 0, score = 0.4038025931142952; 
[10:22:35] Iter 300; Sample 0, score = 0.3450437285302818; 
[10:22:37] Iter 400; Sample 0, score = 0.30451169662313754; 
[10:22:39] Iter 500; Sample 0, score = 0.2747490970224845; 
[10:22:41] Iter 600; Sample 0, score = 0.25168115713944844; 
[10:22:43] Iter 700; Sample 0, score = 0.23333456700279837; 
[10:22:45] Iter 800; Sample 0, score = 0.2182130401657076; 
[10:22:47] Iter 900; Sample 0, score = 0.20563296666317543; 
[10:22:49] Iter 999; Sample 0, score = 0.19491068344038848; 


In [151]:
(model.predict(X_test) * (~np.isnan(y_test_nan))).sum(axis=0) / (~np.isnan(y_test_nan)).sum(axis=0)

array([0.50206699, 0.5017048 , 0.49735283, 0.50005686, 0.50266285,
       0.50213899, 0.50314733, 0.50159522, 0.49943213, 0.5041649 ])

In [153]:
cp.where(~np.isnan(y_test_nan), y_test_nan, 0).sum(axis=0) / (~np.isnan(y_test_nan)).sum(axis=0)

TypeError: Unsupported type <class 'numpy.ndarray'>

In [145]:
from sklearn.metrics import roc_auc_score
roc_auc_score(y_test_nan[:, 0][~np.isnan(y_test_nan[:, 0])], model.predict(X_test)[:, 0][~np.isnan(y_test_nan[:, 0])])

0.993794985643834

## Inference

#### Prediction can be done via calling the .predict method

In [7]:
%%time
preds = model.predict(X_test)

preds.shape

CPU times: user 940 ms, sys: 757 ms, total: 1.7 s
Wall time: 2.06 s


(50000, 10)

In [8]:
preds

array([[-241.72522  , -147.04788  , -279.85645  , ..., -141.56766  ,
        -213.81375  , -235.54959  ],
       [-110.30279  , -112.996796 ,  -60.784744 , ..., -128.7514   ,
        -119.637596 ,  -21.854822 ],
       [ -32.4901   ,  -55.91898  ,  145.11751  , ...,   19.574871 ,
         -20.344044 , -206.56976  ],
       ...,
       [ -75.05906  ,  134.71219  ,   78.33846  , ...,  224.71863  ,
          38.28656  ,   16.880033 ],
       [  -4.0309997,  143.80292  ,  250.64313  , ...,  154.68509  ,
         180.606    ,  212.36513  ],
       [ -20.445889 ,   34.293472 ,  168.47072  , ...,   94.26082  ,
          21.152784 ,    3.533686 ]], dtype=float32)

#### Prediction for certan iterations can be done via calling the .predict_staged method

In [9]:
%%time
preds = model.predict_staged(X_test, iterations=[100, 300, 500])

preds.shape

CPU times: user 312 ms, sys: 382 ms, total: 694 ms
Wall time: 739 ms


(3, 50000, 10)

#### Tree leaves indicies prediction for certan iterations can be done via calling the .predict_leaves method

In [10]:
%%time
preds = model.predict_leaves(X_test, iterations=[100, 300, 500])

preds.shape

CPU times: user 15.8 ms, sys: 3.96 ms, total: 19.8 ms
Wall time: 20.9 ms


(3, 50000, 1)

In [11]:
preds.T[0]

array([[11, 21,  7],
       [54, 46, 18],
       [32, 46, 53],
       ...,
       [54, 53, 10],
       [27, 46, 15],
       [60, 46, 18]], dtype=uint32)

#### Feature importances

In [12]:
model.get_feature_importance()

array([  43.,   45.,   36.,   49.,   63.,   47., 5721.,   48.,   39.,
         66.,   38.,   42.,   50.,   53.,   33., 5873., 5508.,   44.,
         32., 5535.,   32.,   37.,   50.,   70.,   34.,   44.,   46.,
         47.,   47.,   47.,   54.,   49.,   45.,   45.,   35.,   45.,
       6070.,   31.,   47.,   38.,   49.,   49.,   51.,   42.,   54.,
         67.,   50.,   58.,   42.,   51.,   50.,   61., 5875.,   36.,
         58.,   62.,   44.,   21.,   27.,   35.,   47.,   39.,   45.,
         44.,   48.,   45.,   46.,   37.,   54.,   55.,   54.,   38.,
         38.,   48.,   34.,   54.,   60.,   37.,   36.,   55.,   61.,
         40.,   47.,   35.,   63.,   26., 5635., 3557.,   49., 5737.,
         47., 6077.,   38.,   45.,   43.,   45.,   35.,   45.,   35.,
         57.], dtype=float32)

#### The trained model can be saved as pickle for inference

In [13]:
joblib.dump(model, '../data/temp_model.pkl')

new_model = joblib.load('../data/temp_model.pkl')
new_model.predict(X_test)

array([[-241.72522  , -147.04788  , -279.85645  , ..., -141.56766  ,
        -213.81375  , -235.54959  ],
       [-110.30279  , -112.996796 ,  -60.784744 , ..., -128.7514   ,
        -119.637596 ,  -21.854822 ],
       [ -32.4901   ,  -55.91898  ,  145.11751  , ...,   19.574871 ,
         -20.344044 , -206.56976  ],
       ...,
       [ -75.05906  ,  134.71219  ,   78.33846  , ...,  224.71863  ,
          38.28656  ,   16.880033 ],
       [  -4.0309997,  143.80292  ,  250.64313  , ...,  154.68509  ,
         180.606    ,  212.36513  ],
       [ -20.445889 ,   34.293472 ,  168.47072  , ...,   94.26082  ,
          21.152784 ,    3.533686 ]], dtype=float32)